In [22]:
import pandas as pd
import numpy as np

# check scikit-learn version
import sklearn

# check scikit-learn version
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import h2o
from h2o.automl import H2OAutoML

In [2]:
train = pd.read_csv('hospital_train.csv')
test = pd.read_csv('hospital_test.csv')

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.11+9-LTS-194, mixed mode)
  Starting server from C:\Users\Mary\AppData\Local\Programs\Python\Python37\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Mary\AppData\Local\Temp\tmpn673ef32
  JVM stdout: C:\Users\Mary\AppData\Local\Temp\tmpn673ef32\h2o_Mary_started_from_python.out
  JVM stderr: C:\Users\Mary\AppData\Local\Temp\tmpn673ef32\h2o_Mary_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_Mary_i0o58c
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.938 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
columnas = [a for a in h2train.columns if a != '17'][1:]
x = columnas
y = '17'
print(columnas)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16']


In [6]:
automl = H2OAutoML(max_models=50, seed=42, max_runtime_secs=300)
automl.train(x=x, y=y, training_frame=h2train)

AutoML progress: |
17:05:00.490: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [7]:
print('[INFO] Models leader board:')
leader_board = automl.leaderboard
leader_board.head()

[INFO] Models leader board:


model_id,mean_per_class_error,logloss,rmse,mse,auc,aucpr
StackedEnsemble_BestOfFamily_AutoML_20210625_170500,0.761876,1.53251,0.73371,0.53833,nan,nan
StackedEnsemble_AllModels_AutoML_20210625_170500,0.762556,1.53069,0.733297,0.537725,nan,nan
GBM_grid__1_AutoML_20210625_170500_model_1,0.766904,1.54919,0.742601,0.551456,nan,nan
GBM_4_AutoML_20210625_170500,0.775035,1.90248,0.839201,0.704259,nan,nan
DeepLearning_1_AutoML_20210625_170500,0.775161,1.57435,0.74023,0.547941,nan,nan
GBM_2_AutoML_20210625_170500,0.779215,1.82233,0.823414,0.67801,nan,nan
DeepLearning_grid__1_AutoML_20210625_170500_model_1,0.781459,1.63554,0.734329,0.539239,nan,nan
GBM_5_AutoML_20210625_170500,0.784926,1.98739,0.855716,0.732249,nan,nan
GBM_1_AutoML_20210625_170500,0.785609,1.84985,0.828658,0.686674,nan,nan
DeepLearning_grid__2_AutoML_20210625_170500_model_1,0.797111,1.62677,0.754353,0.569048,nan,nan


In [12]:
predictions = automl.leader.predict(h2test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [17]:
test.rename(columns={"Unnamed: 0" : "id"}, inplace = True)

In [18]:
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
1,208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
2,305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
3,266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
4,13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133739,318155,18,d,13,Y,6,anesthesia,Q,B,3.0,123269,25.0,Urgent,Minor,4,51-60,5089.0
133740,144850,27,a,7,Y,2,gynecology,S,C,2.0,1293,8.0,Emergency,Moderate,3,61-70,6713.0
133741,180676,16,c,3,Z,3,gynecology,R,A,3.0,112921,5.0,Trauma,Minor,3,31-40,5326.0
133742,39933,28,b,11,X,4,gynecology,R,F,2.0,585,2.0,Trauma,Extreme,2,31-40,7072.0


In [21]:
test['columna'] = predictions.as_data_frame()

ValueError: Wrong number of items passed 12, placement implies 1

In [13]:

columnasentrega = ["id","17"]
entrega = test["17"]
test = test[columnasentrega]

ValueError: Wrong number of items passed 12, placement implies 1